In [1]:
import laspy
import numpy as np
from scipy.spatial import KDTree
from user_inputs import laz_file_path, clip_folder_path
from user_inputs import search_radius, nn_threshold, bin_count_nn
from helper_functions import generate_bins, fraction_of_trues
import time
from tqdm import tqdm

In [2]:
laz_file_name = laz_file_path.split("\\")[-1].split(".")
clip_laz_file_name = laz_file_name[0] + "_clipped_8th_part." + laz_file_name[1]
clip_laz_path = clip_folder_path + "\\" + clip_laz_file_name
clip_laz_path

'D:\\Geomatics\\thesis\\RefineNet\\data\\AHN\\clipped\\C_37EN2_clipped_8th_part.LAZ'

In [3]:
laz = laspy.read(clip_laz_path)

In [4]:
# number of points
print(f"number of points: {laz.header.point_count}")

number of points: 18742173


In [5]:
# printing out all the available dimensions in clipped laz file
for dimension in laz.point_format.dimensions:
    print(dimension.name)
print("\nunique classifications in clipped region: ", np.unique(laz.classification))

X
Y
Z
intensity
return_number
number_of_returns
scan_direction_flag
edge_of_flight_line
classification
synthetic
key_point
withheld
scan_angle_rank
user_data
point_source_id
gps_time
Amplitude
Reflectance
Deviation

unique classifications in clipped region:  [ 1  2  6  9 26]


**Neighbourhood search steps**
- making a kd tree (first load the points)
- finding the nearest points within 0.5 m radius
    - we can first build np array (or pandas dataframe) with all the required points, classification (labels), rgb(?), intensities
    - then use np.where to find out the distances and indexes of the neighbour points
    - filter only those within 0.5m and look for their classifications
    - give confidence scores to points
    - add extra dimension to laz file

In [6]:
# loading the xyz values
xyz = laz.xyz.copy()
print("xyz shape: ", xyz.shape)
# build a kd tree
tree = KDTree(xyz)

xyz shape:  (18742173, 3)


In [7]:
# loading the classifications of points
labels = laz.classification
print("labels shape: ", labels.shape)

labels shape:  (18742173,)


### **Binning the data to take off load from kd tree**

- number of bins is a hyper-parameter, can be modified in `user_inputs.py`

In [8]:
# binning for neighbourhood search, nobebook crashing when given all the points
num_points = laz.header.point_count
nn_bins = generate_bins(number=num_points, bin_count=bin_count_nn)
print("num_points: ", num_points)
print("number of bins: ", bin_count_nn)
print(nn_bins)

num_points:  18742173
number of bins:  40
[[0, 468554], [468554, 937109], [937109, 1405663], [1405663, 1874217], [1874217, 2342772], [2342772, 2811326], [2811326, 3279880], [3279880, 3748435], [3748435, 4216989], [4216989, 4685543], [4685543, 5154098], [5154098, 5622652], [5622652, 6091206], [6091206, 6559761], [6559761, 7028315], [7028315, 7496869], [7496869, 7965424], [7965424, 8433978], [8433978, 8902532], [8902532, 9371086], [9371086, 9839641], [9839641, 10308195], [10308195, 10776749], [10776749, 11245304], [11245304, 11713858], [11713858, 12182412], [12182412, 12650967], [12650967, 13119521], [13119521, 13588075], [13588075, 14056630], [14056630, 14525184], [14525184, 14993738], [14993738, 15462293], [15462293, 15930847], [15930847, 16399401], [16399401, 16867956], [16867956, 17336510], [17336510, 17805064], [17805064, 18273619], [18273619, 18742173]]


#### *Looping throung all the bins sequentially and calculate confidences*  
- **Get all the neighbouring points (including itself) for a given point**
- seperate the current point from the neighbours
- get classification of the current point
- get classification of the neighbour points
- **Calculate confidences**
    - **count the percentage of neighbour points are of same classification**
    - **points with neighbours < threshold => we would be giving `0` confidence.. should be altered later**

**Confidence = (label_match_count) / (neighbour_count)**
1. get match count
2. get neighbour count
3. calculate confidence

**After**
1. get threshold mask
2. less than 5 threshold number of neighbours, give confidence of `0`

In [9]:
confidence_scores = []
count = 0
point_count = 0
for bin in tqdm(nn_bins, desc="Processing bins"):

    # if count != 15:
    #     count += 1
    #     continue
    # else:
    #     count += 1

    # the query points numbers and their labels    
    start, end = bin[0], bin[1]
    pts = np.arange(start, end, step=1)
    pts_vector = pts.reshape((-1, 1))
    pts_labels = np.array(labels[pts])
    pts_labels = pts_labels.reshape((-1, 1))  # reshaping it to a column vector -> easy to make comparison with neighbours

    # neighbours information of query points
    nn = tree.query_ball_point(xyz[start:end, :], r=0.5, p=2, workers=-1, return_sorted=True)

    #   # should vectorize calculations so find the query point with max number of elements
    max_length = max(len(lst) for lst in nn)
    nn_np_padded = np.array([lst + [np.nan] * (max_length - len(lst)) for lst in nn], dtype=object)  # self points should be removed
    nn_np_padded_float = nn_np_padded.astype(float)  # self points should be removed

    # print("pts_vector[250]: ", pts_vector[250])
    # print("nn_np_padded[250]: ", nn_np_padded[250])

    # remove the search point itself from the neighbours by creating a mask
    self_mask = (pts_vector == nn_np_padded_float)
    nn_np_padded[self_mask] = np.nan
    nn_np_padded_float[self_mask] = np.nan

    # print("nn_np_padded[250]: ", nn_np_padded[250])

    # flattening the neighbours array 
    # so that it is easy to pass to labels and get the classificaitons
    nn_np_padded_flat = nn_np_padded.reshape((1, -1))
    nn_np_padded_float_flat = nn_np_padded_float.reshape((1, -1))
    # nanMask ## True when the element is nan
    nn_np_padded_float_flat_nanMask = np.isnan(nn_np_padded_float_flat)
    nn_np_padded_Mask = ~np.isnan(nn_np_padded_float)

    # replacing all the nan's with -1 index as a placeholder, later we can use mask again to filter them out 
    nn_np_padded_flat[nn_np_padded_float_flat_nanMask] = -1  # replacing all the nan's with -1, now we can use this variable to get neighbours classifications
    nn_labels_flat_pHold = labels[list(nn_np_padded_flat[0])]
    nn_labels_flat_pHold = np.array(nn_labels_flat_pHold)
    # nn_labels_flat = nn_labels_flat_pHold *  ~nn_np_padded_float_flat_nanMask
    nn_labels_flat = nn_labels_flat_pHold.copy()
    nn_labels_flat_float = nn_labels_flat.astype(float)
    nn_labels_flat_float[nn_np_padded_float_flat_nanMask[0]] = np.nan

    # 2d array => no more flattened
    nn_labels_float = nn_labels_flat_float.reshape((nn_np_padded.shape[0], -1))


    # 1. compare pts_labels with nn_labels
    match_pts_nn_bool = (pts_labels == nn_labels_float)
    match_pts_nn_num = np.sum(match_pts_nn_bool, axis=1, keepdims=True) # number of points that match

    # 2. nn_count, create nn mask to filter out the points which are not surrounded by many points
    nn_count = np.sum(nn_np_padded_Mask, axis=1, keepdims=True)
    nn_threshold_mask = (nn_count < nn_threshold)

    # 3. calculating the confidences
    confidences = match_pts_nn_num / nn_count 
    confidences[nn_threshold_mask] = 0
    
    # 4. combining all the bins together
    confidence_scores.append(confidences)

    # print("start, end: ", start, end)
    # print("pts_vector.shape: ", pts_vector.shape)
    # print("pts_vector: ", pts_vector)
    # print("nn.shape: ", nn.shape)
    # print()


confidence_scores = np.vstack(confidence_scores)
print(confidence_scores.shape)    

Processing bins:   0%|          | 0/40 [00:00<?, ?it/s]C:\conda_tmp\ipykernel_25928\394567318.py:67: RuntimeWarning: invalid value encountered in divide
  confidences = match_pts_nn_num / nn_count
Processing bins: 100%|██████████| 40/40 [03:13<00:00,  4.83s/it]

(18742173, 1)


In [10]:
confidence_scores

array([[1. ],
       [1. ],
       [1. ],
       ...,
       [1. ],
       [0.9],
       [1. ]])

## Useless

In [ ]:
confidence_scores = []
count = 0
point_count = 0

for bin in tqdm(nn_bins, desc="Processing bins"):

    # the query points numbers and their labels 
    start, end = bin[0], bin[1]
    pts = np.arange(start, end, step=1)
    pts_vector = pts.reshape((-1, 1))
    pts_labels = np.array(labels[pts])
    pts_labels = pts_labels.reshape((-1, 1))  # reshaping it to a column vector -> easy to make comparison with neighbours

    point_count += pts_vector.shape[0]


    # # neighbours information of query points
    # nn = tree.query_ball_point(xyz[start:end, :], r=0.5, p=2, workers=-1, return_sorted=True)

    # #   # should vectorize calculations so find the query point with max number of elements
    # max_length = max(len(lst) for lst in nn)
    # nn_np_padded = np.array([lst + [np.nan] * (max_length - len(lst)) for lst in nn], dtype=object)  # self points should be removed
    # nn_np_padded_float = nn_np_padded.astype(float)  # self points should be removed

    # print("pts_vector[250]: ", pts_vector[250])
    # print("nn_np_padded[250]: ", nn_np_padded[250])

    # # remove the search point itself from the neighbours by creating a mask
    # self_mask = (pts_vector == nn_np_padded_float)
    # nn_np_padded[self_mask] = np.nan
    # nn_np_padded_float[self_mask] = np.nan

    # print("nn_np_padded[250]: ", nn_np_padded[250])

    # # flattening the neighbours array 
    # # so that it is easy to pass to labels and get the classificaitons
    # nn_np_padded_flat = nn_np_padded.reshape((1, -1))
    # nn_np_padded_float_flat = nn_np_padded_float.reshape((1, -1))
    # # nanMask ## True when the element is nan
    # nn_np_padded_float_flat_nanMask = np.isnan(nn_np_padded_float_flat)
    # nn_np_padded_Mask = ~np.isnan(nn_np_padded_float)

    # # replacing all the nan's with -1 index as a placeholder, later we can use mask again to filter them out 
    # nn_np_padded_flat[nn_np_padded_float_flat_nanMask] = -1  # replacing all the nan's with -1, now we can use this variable to get neighbours classifications
    # nn_labels_flat_pHold = labels[list(nn_np_padded_flat[0])]
    # nn_labels_flat_pHold = np.array(nn_labels_flat_pHold)
    # # nn_labels_flat = nn_labels_flat_pHold *  ~nn_np_padded_float_flat_nanMask
    # nn_labels_flat = nn_labels_flat_pHold.copy()
    # nn_labels_flat_float = nn_labels_flat.astype(float)
    # nn_labels_flat_float[nn_np_padded_float_flat_nanMask[0]] = np.nan

    # # 2d array => no more flattened
    # nn_labels_float = nn_labels_flat_float.reshape((nn_np_padded.shape[0], -1))


    # # 1. compare pts_labels with nn_labels
    # match_pts_nn_bool = (pts_labels == nn_labels_float)
    # match_pts_nn_num = np.sum(match_pts_nn_bool, axis=1, keepdims=True) # number of points that match

    # # 2. nn_count, create nn mask to filter out the points which are not surrounded by many points
    # nn_count = np.sum(nn_np_padded_Mask, axis=1, keepdims=True)
    # nn_threshold_mask = (nn_count < nn_threshold)

    # # 3. calculating the confidences
    # confidences = match_pts_nn_num / nn_count 
    # confidences[nn_threshold_mask] = 0
    


    # print("start, end: ", start, end)
    # print("pts_vector.shape: ", pts_vector.shape)
    # print("pts_vector: ", pts_vector)
    # print("nn.shape: ", nn.shape)
    # # print()


    # 
print(f"point_count: {point_count}")
    

In [11]:
laz.add_extra_dim(laspy.ExtraBytesParams(
    name="confidence",
    type=np.float64,
    description="Confidence of points"
))

In [12]:
# printing out all the available dimensions in clipped laz file
for dimension in laz.point_format.dimensions:
    print(dimension.name)


X
Y
Z
intensity
return_number
number_of_returns
scan_direction_flag
edge_of_flight_line
classification
synthetic
key_point
withheld
scan_angle_rank
user_data
point_source_id
gps_time
Amplitude
Reflectance
Deviation
confidence


In [13]:
laz.confidence = confidence_scores.reshape((confidence_scores.shape[0], ))

In [14]:
laz.confidence

array([1. , 1. , 1. , ..., 1. , 0.9, 1. ])

## outputting the file with confidences

In [15]:
confidence_clip_laz_file_name = laz_file_name[0] + "_clipped_8th_part_confidences." + laz_file_name[1]
confidence_clip_laz_path = clip_folder_path + "\\" + confidence_clip_laz_file_name
confidence_clip_laz_path

'D:\\Geomatics\\thesis\\RefineNet\\data\\AHN\\clipped\\C_37EN2_clipped_8th_part_confidences.LAZ'

In [16]:
# outputting the file
# laz_with_confidence = 
# Create a new LAS/LAZ file for writing
laz_with_confidence = laspy.file.File(confidence_clip_laz_path, mode="w", header=laz.header)

# Copy existing header
laz_with_confidence.header = laz.header
laz_with_confidence.write_points(laz.points)
laz_with_confidence.close()

LaspyException: You are using laspy 2.0, which has several improvements over 1.x
            but with several breaking changes.
            To stay on laspy 1.x: `pip install laspy<2.0.0`
            
            In short:
              - To read a file do: las = laspy.read('somefile.laz')
              - To create a new LAS data do: las = laspy.create(point_format=2, file_version='1.2')
              - To write a file previously read or created: las.write('somepath.las')
            See the documentation for more information about the changes https://laspy.readthedocs.io/en/latest/

In [20]:
with laspy.open(confidence_clip_laz_path, mode="w", header=laz.header) as writer:
        # for points in laz.chunk_iterator(100000):
    writer.write_points(laz.points)


In [18]:
laz.header.point_count

18742173